In [1]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path
import numpy as np
import cv2
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import tqdm

sys.path.append('fbsource/fbcode/surreal/')
from maploc.data.holicity import Projection as ProjectionHolicity, prepare_views
from maploc.utils.geo import Projection
from maploc.osm.tiling import prepare_tiles, plot_locations
from maploc.osm.parser import match_to_group, Patterns, Groups
from maploc.utils.viz_2d import plot_images, plot_keypoints, add_text, save_plot
from maploc.utils.geometry import Pose
from maploc.utils.geo import BoundaryBox

root = Path('buckets/llp_datasets/holicity/')
root = Path('./data/holicity_v2')

In [2]:
proj = Projection("epsg:27700")
dump = prepare_views(root, proj)

In [69]:
tile_radius = 96
ppm = 2
out_dir = Path('./data/holicity_v2_dump')
out_dir.mkdir(exist_ok=True, parents=True)
osm_path = Path("./data/osm/greater-london-latest-cut.osm")
osm, tiling = prepare_tiles(dump["views"], proj, out_dir, tile_radius, ppm, viz=True, skip_write=False, osm_path=osm_path)

from maploc.utils.io import write_json
dump_path = out_dir / "dump.json"
dump_path.parent.mkdir(parents=True, exist_ok=True)
write_json(dump_path, {**dump, **tiling})

In [ ]:
from maploc.osm.analysis import plot_osm_histograms
plot_osm_histograms(osm, fontsize=3.5, dpi=200)

In [ ]:
from maploc.osm.analysis import plot_sankey_hierarchy
plot_sankey_hierarchy(osm);

# Create data split

In [123]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from  maploc.utils.geo import BoundaryBox
from maploc.utils.io import write_json
import json

margin = 96*2

# read dump file
root = Path('./data/holicity_v2_dump/')
with open(root / 'dump.json', 'r') as fp:
    dump = json.load(fp)
names = list(dump["views"])
locations = np.array([dump["views"][n]["t_c2w"][:2] for n in names])
bbox_all = BoundaryBox(locations.min(0), locations.max(0))

med = np.percentile(locations, 70, 0)
size = 220
bbox_val = BoundaryBox(med-size, med+size)
is_val = bbox_val.contains(locations)
split = {
    'train': [names[i] for i in np.where(~is_val)[0]],
    'val': [names[i] for i in np.where(is_val)[0]],
}
print(is_val.mean(), len(split['train']), len(split['val']))
write_json(root / f'splits_{size}m.json', split)

def draw_bbox(min_, max_, color):
    plt.gca().add_patch(
        mpl.patches.Rectangle(
            min_, *(max_ - min_), lw=1, ec=color, fc="none"
        )
    )
plt.scatter(*locations.T, s=0.1)
draw_bbox(bbox_all.min_, bbox_all.max_, 'r')
draw_bbox(bbox_all.min_ - margin, bbox_all.max_ + margin, 'k')
draw_bbox(bbox_val.min_, bbox_val.max_, 'g')
plt.gca().set_aspect("equal")
plt.scatter(*med, s=10, color='red')

# Check projections

In [ ]:
names = list(views.keys())
name1, name2 = np.random.RandomState(0).choice(names, 2)
xy1 = np.array(views[name1]["t_c2w"])[:2]
xy2 = np.array(views[name2]["t_c2w"])[:2]
geo1 = np.array(views[name1]["gps"])
geo2 = np.array(views[name2]["gps"])
d = np.linalg.norm(xy1 - xy2)
d2 = np.linalg.norm(proj2.project(geo1) - proj2.project(geo2))
print(d, d2)


In [ ]:
names = list(views.keys())
name1, name2 = np.random.RandomState(0).choice(names, 2)
xy1 = np.array(views[name1]["t_c2w"])[:2]
geo1 = np.array(views[name1]["gps"])
geo2 = geo1 + np.array([1e-3, 1e-3])
xy2 = proj.project(geo2)

d = np.linalg.norm(xy1 - xy2)
d2 = np.linalg.norm(proj2.project(geo1) - proj2.project(geo2))
print(d, d2)
